In [1]:
import os
import json
import wget
import pandas as pd
from wordle_game import WordleGame

In [2]:
# Load relevant files
source_file = 'dictionary.json'
myurl = f"https://raw.githubusercontent.com/matthewreagan/WebstersEnglishDictionary/master/{source_file}"

if not(os.path.exists(source_file)):
    wget.download(myurl , os.getcwd())

mod_dict_file = 'words_dictionary_length.json'

if not(os.path.exists(mod_dict_file)):

    with open(source_file,'r', encoding='utf-8') as word_dict_file:
        WORD_DICT_ORIG = json.load(word_dict_file)

    WORD_DICT = {}

    for key in WORD_DICT_ORIG.keys():
        WORD_DICT[key] = len(key)

    with open(mod_dict_file, 'w') as f:
        json.dump(WORD_DICT, f)
else:
    with open(mod_dict_file) as word_len_dict_file:
        WORD_DICT = json.load(word_len_dict_file)

In [3]:
def random_solver(newGame:WordleGame, print_guess = False):
    guess_list = []
    current_guess = newGame._sample_random_word()
    guess_list.append(current_guess)
    while current_guess != newGame.target_word:
        newGame._update_guess_set(current_guess, print_guess)
        current_guess = newGame._sample_random_word()
        print(f"Current guess is:{current_guess}")
        guess_list.append(current_guess)

    return current_guess, guess_list

In [4]:
def letter_frequency_solver(newGame:WordleGame, letter_freuqency: pd.DataFrame, print_guess = False):
    # guess_list = []
    # current_guess = newGame._sample_random_word()
    # guess_list.append(current_guess)
    pass

In [5]:
newGame = WordleGame(5,WORD_DICT)
newGame._set_target_word('evoke')
newGame.target_word

'evoke'

In [6]:
random_solver(newGame)

Current guess is:spyne
Current guess is:braze
Current guess is:exode
Current guess is:eloge
Current guess is:evoke


('evoke', ['cumic', 'spyne', 'braze', 'exode', 'eloge', 'evoke'])